In [1]:
# generate the symmetry-corrected indices for move-based convolution
import sys
import numpy as np
sys.path.append('..')
from neural import generate_all_moves_by_index, move_convolution_indices
#generate_all_moves_by_index()
all_inds, num_coeffs = move_convolution_indices()
num_coeffs -= 20 # the first 10 in the above function are biases, don't need them, the next 10 are self coeffs
num_biases = 10
print(num_coeffs)
num_fields = 7*7

31


In [2]:
# test the coefficient generation logic the naive way
cell = 22

In [3]:
from neural import to_pair
cell = cell+1
tmp = all_inds[cell]
a = np.zeros([7,7])
for (ind, coeff) in tmp[1:]:
    pair = to_pair(ind)
    a[pair[0],pair[1]] = coeff - 9

print(to_pair(cell))
print(a)

(2, 3)
[[  0.   0.  36.   0.  36.   0.   0.]
 [  0.  37.   0.   0.   0.  37.   0.]
 [  0.   0.   0.   9.   0.   0.   0.]
 [  0.  38.   0.   0.   0.  38.   0.]
 [  0.   0.  39.   0.  39.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]]


In [4]:
def vec_to_board(vec):
    board = np.reshape(vec,[7,7])
    return board

In [5]:
# convert that dict into a 3d-tensor,
# [in_field, out_field, coeff_ind]

conv_map = np.zeros([num_fields, num_fields, num_coeffs])
bias_map = np.zeros([num_fields, num_biases])
for cell in range(num_fields):
    tmp = all_inds[cell]
    bias_map[cell, tmp[0][1]] = 1
    #print(cell,tmp[0])
    for (ind, coeff) in tmp[1:]:
        if coeff>=20:
            pair = to_pair(ind)
            conv_map[cell, ind, coeff-20] = 1      

In [6]:
# Now let's do the convolution in Tensorflow
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

In [7]:
conv_mapping = tf.constant(conv_map,dtype=tf.float32, name = 'conv_mapping')  
bias_mapping = tf.constant(bias_map,dtype=tf.float32, name = 'bias_mapping')  

In [8]:
def convolve_by_moves(in_fields, mask, this_conv_coeffs, biases): 
    '''
    in_fields is a batch of tensors [batch_size, num_fields, num_channels]
    mask is a tensor of 0s and 1s [num_fields]
    so conv_coeffs must have size [num_coeffs, channels_in, channels_out]
    '''
    
    tmp = tf.tensordot(conv_mapping, this_conv_coeffs, [[2],[0]])
    #print(sess.run([tf.shape(tmp), tf.shape(in_fields)]))
    tmp2 = tf.matmul(tf.cast(in_fields,tf.float32), tmp)
    out = tf.multiply(tmp2, tf.cast(mask,tf.float32))
    # TODO: biases must also be symmetry-corrected!
    return out + biases

# this wrapper just defines the Variables, to be replaced by a Keras wrapper
 # approx avg number of inputs is 6 or so, so normalize init weights accordingly
def convolve_by_moves_with_coeffs(in_fields, mask, 
                      wgt_init = tf.truncated_normal(shape=[num_coeffs],stddev = 1/2.5,
                                                     dtype=tf.float32)):
    this_conv_coeffs = tf.Variable(wgt_init)
    # TODO: biases must also be symmetry-corrected!
    biases = tf.Variable(np.zeros(num_fields), dtype = tf.float32) 
    return convolve_by_moves(in_fields, mask, this_conv_coeffs, biases)

In [9]:
def ch_convolve_by_moves(in_fields, mask, this_conv_coeffs, biases, sess=None): 
    '''
    in_fields is a batch of tensors [batch_size, num_fields, num_channels]
    mask is a tensor of 0s and 1s [num_fields]
    so conv_coeffs must have size [num_coeffs, channels_in, channels_out]
    '''
    input_dim = in_fields.get_shape().as_list()
    if len(input_dim) == 2: # just batch and fields
        in_fields = tf.expand_dims(in_fields,2) # add the channel dimension
    # inputs conv_mapping[i,j,k], this_conv_coeffs[k,m,l], output tmp[i,j,m,l]
    tmp = tf.tensordot(conv_mapping, this_conv_coeffs, [[2],[0]]) 
    #print(sess.run([tf.shape(tmp), tf.shape(in_fields)]))
    # in_fields[b,j,m], output is tmp2[b,i,l]
    tmp2 = tf.tensordot(tf.cast(in_fields,tf.float32), tmp, [[1,2],[1,2]])
    
    # bias_mapping[i,k], biases[k,l], bias_term should be [b,i,l] but in this line just get [i,l]: 
    tmp_bias = tf.tensordot(bias_mapping, biases,[[1],[0]])

    out = tmp2 + tf.expand_dims(tmp_bias, 0) # use broadcasting to add biases to each batch
    
    if mask is not None:
        # mask is [b,i], batches x num_fields, need to apply to all channels of output - use broadcasting
        if len(mask.get_shape()) == 2:
            mask = tf.expand_dims(mask, 2)
        out = tf.multiply(out, tf.cast(mask,tf.float32))
    return out

# this wrapper just defines the Variables, to be replaced by a Keras wrapper
 # approx avg number of inputs is 6 or so, so normalize init weights accordingly
def ch_convolve_by_moves_with_coeffs(in_fields, mask, out_channels, wgt_init = None, sess=None):
    in_channels = in_fields.get_shape().as_list()[-1]
    #print(sess.run(tf.shape(in_fields)[-1]))
    #print(in_fields.get_shape().as_list())
    if not wgt_init:
        wgt_init = tf.truncated_normal(shape=[num_coeffs,in_channels, out_channels],stddev = 1/2.5,
                                                     dtype=tf.float32)
    this_conv_coeffs = tf.Variable(wgt_init)
    
    biases = tf.Variable(np.zeros([num_biases, out_channels]), dtype = tf.float32)
    return ch_convolve_by_moves(in_fields, mask, this_conv_coeffs, biases)
    

In [10]:
## Show all coefficient indices
# with tf.Session() as sess:
#     for ind in range(49):
#         in_fields_np = np.zeros([2,num_fields])
#         in_fields_np[0,3] = 1
#         in_fields_np[0,5] = 1
#         in_fields_np[1,ind] = 1
#         in_fields = tf.constant(np.array(in_fields_np))

#         mask = tf.constant(np.ones(in_fields_np[0].shape))
#         out = convolve_by_moves(in_fields, mask, np.ones(num_coeffs))
#         out_2 = convolve_by_moves(out, mask, np.ones(num_coeffs))
#         sess.run(tf.global_variables_initializer())
#         print(vec_to_board(sess.run(out_2)[1]))

In [11]:
def get_random_index(x, ind, batch_size, sess = None):
    '''
    x: [batch_size, num_fields]
    ind: [batch_size,2] # 2 random indexes (me and opponent) I want to grab in the corresponding row
    '''
    #
    batch_ind = tf.constant(np.array(range(batch_size))[:,None])
    batch_nums = tf.cast(batch_ind, tf.int32)
    ind =tf.cast(ind, tf.int32)
    ind1 = tf.slice(ind,[0,0],[-1,1])
    ind2 = tf.slice(ind,[0,1],[-1,1])
    ind_ext1 = tf.concat([batch_nums,ind1],1)
    ind_ext2 = tf.concat([batch_nums, ind2],1)
    out1 =tf.expand_dims(tf.gather_nd(x,ind_ext1),2)
    out2 =tf.expand_dims(tf.gather_nd(x,ind_ext2),2)
    out = tf.concat([out1,out2],2)
    print(sess.run(tf.shape(out)))
    return out

def conv_stack(inputs, num_layers, sess = None):
    '''
    in_fields: [batch_size, num_fields]
    num_layers: int
    my_pos: [batch_size, 1]
    other_pos: [batch_size, 1]
    '''
    in_fields = tf.slice(inputs,[0,0],[-1,num_fields])
    player_pos = tf.slice(inputs,[0,num_fields],[-1,2])
#     print(sess.run(in_fields))
#     print(sess.run(tf.shape(my_pos)))
#     print(sess.run(tf.shape(other_pos)))
    mask = in_fields
    out = tf.expand_dims(in_fields,2) # add the channel dimension
    for _ in range(num_layers):
        out = ch_convolve_by_moves_with_coeffs(out, in_fields, 3, sess=sess)
        
    player_wgt = tf.Variable(tf.truncated_normal(shape =[1,2],dtype=tf.float32))
    #other_wgt = tf.Variable(tf.truncated_normal(shape =[1],dtype=tf.float32))
    
    batch_size = out.shape[0]
    return player_wgt*get_random_index(out, player_pos, batch_size,sess) 
        

In [12]:
# try calling conv_stack
with tf.Session() as sess:
    in_fields_np = np.ones([2,num_fields])
    in_fields_np[0,3] = 0
    in_fields_np[0,5] = 0
    my_pos = np.array([24, 24])
    other_pos =  np.array([33,33])
    inputs_np = np.concatenate([in_fields_np, my_pos[:,None], other_pos[:,None]],
                              1)
    #print(inputs_np.shape)
    inputs =tf.constant(inputs_np)# tf.placeholder(shape =[None, num_fields+2], dtype = tf.float32) #
    #print(sess.run(inputs))
    out = conv_stack(inputs, 5,sess)
    
    dummy = np.array([50,50])[:,None]
    #print(sess.run(get_random_index(inputs, tf.constant(dummy))))
    
    sess.run(tf.global_variables_initializer())
    stack_result = sess.run(out)#, feed_dict={inputs:inputs_np})
    print(stack_result)

[2 3 2]
[[[-95.14674377  -4.39386368]
  [-26.99053001   7.03287792]
  [-61.09555435   0.90047985]]

 [[-93.96670532  -5.07419491]
  [-27.18621826   7.13246584]
  [-59.01132202   0.50100213]]]


In [13]:
# # create a Keras model from the above, to simplify fitting: just using Lambda fails as tf.Variables are not recognized by Keras
# from keras.models import Model, Sequential
# from keras.layers import Input, Lambda

# #model_in = tf.placeholder(tf.float32, shape = (None, 51))
# #model_out = conv_stack(model_in, 5)
# my_fun = Lambda(lambda x: conv_stack(x, 5))
# #my_model = Model(Input(tensor = model_in), outputs = my_fun(model_in))
# my_input = Input(shape = [51])
# my_output = my_fun(my_input)
# my_model = Model(inputs = my_input, outputs = my_output)
# my_model.summary()

In [14]:
# So let's create a custom Keras layer instead

# def ch_convolve_by_moves_with_coeffs(in_fields, mask, out_channels):
#     in_channels = in_fields.get_shape().as_list()[-1]
#     wgt_init = tf.truncated_normal(shape=[num_coeffs,in_channels, out_channels],
#                                     stddev = 1/2.5,
#                                     dtype=tf.float32)
#     this_conv_coeffs = tf.Variable(wgt_init)
#     biases = tf.Variable(np.zeros([num_biases, out_channels]), dtype = tf.float32)
#     return ch_convolve_by_moves(in_fields, mask, this_conv_coeffs, biases)

from keras import backend as K
from keras.engine.topology import Layer
from keras.initializers import TruncatedNormal
import numpy as np

class ConvByMoveLayer(Layer):

    def __init__(self, out_channels, mask = None, **kwargs):
        self.out_channels = out_channels
        self.mask = mask
        super(ConvByMoveLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        if len(input_shape)==2:
            in_channels = 1
        else:
            in_channels = input_shape[2]
            
        init_std = 1/(3*np.sqrt(in_channels*self.out_channels))
            
        self.conv_coeffs = self.add_weight(name='conv_coeffs', 
                                      shape=(num_coeffs,in_channels, self.out_channels),
                                      initializer=TruncatedNormal(stddev = init_std),
                                      trainable=True)
        #print((num_coeffs, in_channels, self.out_channels))
        self.biases = self.add_weight(name='biases', 
                                      shape=(num_biases,self.out_channels),
                                      initializer='zeros',
                                      trainable=True)
        super(ConvByMoveLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return ch_convolve_by_moves(x,self.mask, self.conv_coeffs, self.biases)
    def compute_output_shape(self, input_shape):
        return (input_shape[0], num_fields, self.out_channels)

Using TensorFlow backend.


In [15]:
# from keras.models import Model, Sequential
# from keras.layers import InputLayer, Lambda, Flatten, Dense, merge
# from keras.layers.merge import Concatenate
# from keras import backend as K

# player_pos_one_hot = Input(shape = [49, 2])
# board_state = Input(shape=[49,1])
# mask = board_state
# #tmp1 = K.expand_dims(board_state, 2)# TODO: do this in Keras code
# out = Concatenate()([board_state, player_pos_one_hot])

# out= ConvByMoveLayer(3, mask)(out)
# out = ConvByMoveLayer(5, mask)(out)
# out = ConvByMoveLayer(7, mask)(out)
# #out = Lambda(lambda x: get_random_index(*x), arguments = {batch_size,2})([out,player_pos])
# out = Concatenate()([out, player_pos_one_hot])
# out = Flatten()(out)
# out = Dense(10, activation = 'relu')(out)
# out = Dense(1)(out)

# model = Model(inputs = [player_pos_one_hot, board_state], outputs = out)
# model.summary()
# model.compile(optimizer = 'adam',  loss='mean_squared_error')

In [16]:
# load game simulation data
import glob
import sys
#sys.path.append('./neural')
from data_utils import load_simulation_data
files = glob.glob('../data/ID_x2_1000ms/result_ID*.pickle')
print(files)
depths =load_simulation_data(files)
keys = list(depths.keys())
print(keys)
games = depths[keys[0]]
print(games[0])

['../data/ID_x2_1000ms/result_ID_x2_24.pickle', '../data/ID_x2_1000ms/result_ID_x2_22.pickle', '../data/ID_x2_1000ms/result_ID_x2_17.pickle', '../data/ID_x2_1000ms/result_ID_x2_4.pickle', '../data/ID_x2_1000ms/result_ID_x2_16.pickle', '../data/ID_x2_1000ms/result_ID_x2_8.pickle', '../data/ID_x2_1000ms/result_ID_x2_20.pickle', '../data/ID_x2_1000ms/result_ID_x2_13.pickle', '../data/ID_x2_1000ms/result_ID_x2_5.pickle', '../data/ID_x2_1000ms/result_ID_x2_2.pickle', '../data/ID_x2_1000ms/result_ID_x2_19.pickle', '../data/ID_x2_1000ms/result_ID_x2_11.pickle', '../data/ID_x2_1000ms/result_ID_x2_23.pickle', '../data/ID_x2_1000ms/result_ID_x2_10.pickle', '../data/ID_x2_1000ms/result_ID_x2_12.pickle', '../data/ID_x2_1000ms/result_ID_x2_18.pickle', '../data/ID_x2_1000ms/result_ID_x2_9.pickle', '../data/ID_x2_1000ms/result_ID_x2_6.pickle', '../data/ID_x2_1000ms/result_ID_x2_7.pickle', '../data/ID_x2_1000ms/result_ID_x2_21.pickle', '../data/ID_x2_1000ms/result_ID_x2_1.pickle', '../data/ID_x2_1000m

In [17]:
from sklearn.preprocessing import OneHotEncoder


def prepare_data_for_model(states, score_name = 'simple_score'):
    y = np.array([state[score_name] for state in states])
    board = np.array([list(state['game']) for state in states])
    pos = np.array([list(state['pos']) for state in states])
    print(board.shape,pos.shape, y.shape)
    encoder = OneHotEncoder(49)
    pos_oh = encoder.fit_transform(pos).toarray()

    # now make sure they're the right shape
    player_pos_one_hot_value = np.array( np.concatenate( [pos_oh[:,:49,None],pos_oh[:,49:,None]],2))
    print(player_pos_one_hot_value[0])
    board_full = np.array(np.reshape(board, [board.shape[0],49,1]))

    return board_full, player_pos_one_hot_value, y[:,None]



In [18]:
states = [state for game in games for state in game] 
board_full, player_pos, y = prepare_data_for_model(states)

(984694, 49) (984694, 2) (984694,)
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [19]:
from keras.models import Model, Sequential
from keras.layers import Input, Lambda, Flatten, Dense, Activation
from keras.layers.merge import Concatenate, Add
from keras.layers.normalization import BatchNormalization
from keras import backend as K

player_pos_one_hot = Input(shape = [49, 2])
board_state = Input(shape=[49,1])
mask = board_state
num_features = 3

def ResNetLayerFun(x, num_features = 3, mask = None):
    tmp = BatchNormalization()(x)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    tmp = BatchNormalization()(tmp)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    return Add()([x,tmp])

#tmp1 = K.expand_dims(board_state, 2)# TODO: do this in Keras code
out = Concatenate()([board_state, player_pos_one_hot])
out = ConvByMoveLayer(num_features, mask)(out)
out = ResNetLayerFun(out, num_features, mask)
out = Activation('relu')(out)
out = Concatenate()([out, player_pos_one_hot])
out = Flatten()(out)
out = Dense(10, activation = 'relu')(out)
out = Dense(1)(out)

model = Model(inputs = [player_pos_one_hot, board_state], outputs = out)
model.summary()
model.compile(optimizer = 'adam',  loss='mean_squared_error')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 49, 1)         0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 49, 2)         0                                            
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 49, 3)         0           input_2[0][0]                    
                                                                   input_1[0][0]                    
____________________________________________________________________________________________________
conv_by_move_layer_1 (ConvByMove (None, 49, 3)         309         concatenate_1[0][0]     

In [20]:
#model.fit([player_pos, board_full],y, batch_size = 256, epochs=5, verbose =1)

In [21]:
# Now let's get all those games where tree search actually completed
import numpy as np
from sklearn.preprocessing import OneHotEncoder

complete_states = [state for game in games for state in game if state['score'] == float('inf') or state['score'] == float('-inf')]
print(len(complete_states))
board_full_c, player_pos_c, y_c = prepare_data_for_model(complete_states,'score')
y_c[y_c==float('inf')] = 1
y_c[y_c==float('-inf')] = 0

365201
(365201, 49) (365201, 2) (365201,)
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [22]:
print(set(list(np.reshape(y_c,[-1]))))

{0.0, 1.0}


In [23]:
from keras.models import Model, Sequential
from keras.layers import Input, Lambda, Flatten, Dense, Activation, Dropout
from keras.layers.merge import Concatenate, Add
from keras.layers.normalization import BatchNormalization
from keras import backend as K

player_pos_one_hot = Input(shape = [49, 2])
board_state = Input(shape=[49,1])
mask = board_state
num_features = 8
num_blocks = 8
num_res_modules = 32
drop_rate = 0.2

def BlockFun(x, num_features = 3, mask = None):
    tmp = BatchNormalization()(x)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    tmp = BatchNormalization()(tmp)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    tmp = Dropout(drop_rate)(tmp)
    return tmp

def ResNetLayerFun1(x, num_features = 3, num_blocks=1, mask = None):
    tmp = x
    for _ in range(num_blocks):
        tmp = Add()([x,BlockFun(x, num_features, mask)])
    return tmp

def ResNetLayerFun2(x, num_features = 3, num_blocks=1, mask = None):
    tmp = [x]
    for _ in range(num_blocks):
        tmp.append(BlockFun(x, num_features, mask))
    return Concatenate()(tmp)
    
    

#tmp1 = K.expand_dims(board_state, 2)# TODO: do this in Keras code
out = Concatenate()([board_state, player_pos_one_hot])
out = ConvByMoveLayer(num_features, mask)(out)
for _ in range(num_res_modules):
    out = ResNetLayerFun1(out, num_features, num_blocks, mask)
out = Activation('relu')(out)
out = Concatenate()([out, player_pos_one_hot])
out = Flatten()(out)
out = Dense(10, activation = 'relu')(out)
out = Dense(2, activation = 'softmax')(out)

deep_model = Model(inputs = [player_pos_one_hot, board_state], outputs = out)
deep_model.summary()
deep_model.compile(optimizer = 'adam',  loss='categorical_crossentropy', metrics =['acc'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 49, 1)         0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 49, 2)         0                                            
____________________________________________________________________________________________________
concatenate_3 (Concatenate)      (None, 49, 3)         0           input_4[0][0]                    
                                                                   input_3[0][0]                    
____________________________________________________________________________________________________
conv_by_move_layer_4 (ConvByMove (None, 49, 8)         824         concatenate_3[0][0]     

In [24]:
from keras.utils.np_utils import to_categorical
deep_model.fit([player_pos_c, board_full_c],to_categorical(y_c, num_classes=2), batch_size = 64, epochs=10, verbose =1, validation_split = 0.1)

Train on 328680 samples, validate on 36521 samples
Epoch 1/10
328680/328680 [==============================] - 1162s - loss: 0.3546 - acc: 0.8436 - val_loss: 0.2543 - val_acc: 0.9069
Epoch 2/10
328680/328680 [==============================] - 1131s - loss: 0.2477 - acc: 0.9090 - val_loss: 0.2395 - val_acc: 0.9105
Epoch 3/10
328680/328680 [==============================] - 1122s - loss: 0.2357 - acc: 0.9124 - val_loss: 0.2304 - val_acc: 0.9134
Epoch 4/10
328680/328680 [==============================] - 1125s - loss: 0.2293 - acc: 0.9144 - val_loss: 0.2239 - val_acc: 0.9177
Epoch 5/10
328680/328680 [==============================] - 1119s - loss: 0.2236 - acc: 0.9162 - val_loss: 0.2189 - val_acc: 0.9178
Epoch 6/10
328680/328680 [==============================] - 1126s - loss: 0.2206 - acc: 0.9173 - val_loss: 0.2178 - val_acc: 0.9191
Epoch 7/10
328680/328680 [==============================] - 1118s - loss: 0.2164 - acc: 0.9187 - val_loss: 0.2147 - val_acc: 0.9205
Epoch 8/10
328680/328680 

In [25]:
from collections import namedtuple
from copy import copy

SimpleGame = namedtuple("Simple_game", ["moving_player_pos","other_player_pos", "board"])
move_dict = generate_all_moves_by_index()

def get_legal_moves(game):
    if game.moving_player_pos is None:
        return [m for m in range(49) if game.board[m] == 1]
    else:
        moves = move_dict[game.moving_player_pos]
        return [m for m in moves if game.board[m] ==1 ]

def apply_move(game, move):
    if not move in get_legal_moves(game):
        raise ValueError('Illegal move!')
    new_board = copy(game.board)
    new_board[move] = 0
    other_pos = move
    moving_pos = game.other_player_pos
    return SimpleGame(moving_pos, other_pos, new_board)
    
board = np.ones(49)
board.sum()
my_pos = None
other_pos = None
game = SimpleGame(my_pos, other_pos, board)
game1 = apply_move(game, 0, move_dict)
game2 = apply_move(game1, 1, move_dict)
game3 = apply_move(game2, 14, move_dict)
print(game3)

TypeError: apply_move() takes 2 positional arguments but 3 were given

In [ ]:
# sort all games by number of moves. 


states_by_num_moves = [[] for _ in range(49)]

for state in states:
    moves_made = 49 - state['game'].sum()
    states_by_num_moves[int(moves_made)].append(state)
    
for n in range(49):
    print(n,len(states_by_num_moves[n]))

In [ ]:
# Iteratively populate all non-+-inf values in layer n from evaluating model in layer n+1, then include these into the fitting set
# after each pass, refresh the values for earlier layers

prepared_data = [None for _ in range(49)]
for n in range(2,49):
    prepared_data[n] = prepare_data_for_model( states_by_num_moves[n],'score') # board, player_pos, score
    
# TODO: is my position always first in those dumps???
    
def recursively_fill_scores(board, player_pos, scores, eval_fun):
    new_scores = scores.copy()
    new_scores[scores == float('inf')] =1 
    new_scores[scores == float('-inf')] = 0
    for n, score in enumerate(scores):
        if score not in [float('inf'), float('-inf')]:
            new_scores[n] = get_recursive_score(board[n], player_pos[n], eval_fun)
            
def get_recursive_score(board, player_pos, eval_fun):
    this_game = SimpleGame(board, player_pos[0], player_pos[1])
    moves = get_legal_moves(this_game)
    vals = np.array([1 - eval_fun(apply_move(this_game, move)) for move in moves])
    return vals.max()